# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [34]:
# Initial imports
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import nltk as nltk
from newsapi import NewsApiClient
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Jordan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv("news_api")

In [3]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [4]:
# Fetch the Bitcoin news articles
btc_news = newsapi.get_everything(q="Bitcoin", from_param="2021-06-01")

In [5]:
# Fetch the Ethereum news articles
eth_news = newsapi.get_everything(q="Ethereum",from_param="2021-06-01")

In [6]:
def get_sent_df(topic_news):
    sentiments = []
    
    for article in topic_news["articles"]:
        try:
            text = article["content"]
            date = article["publishedAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
        
            sentiments.append({
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu
            
            })

        except AttributeError:
            pass
    
    df = pd.DataFrame(sentiments)
    cols = ["date", "text", "compound", "positive", "negative", "neutral"]
    df = df[cols]
    
    return df

In [7]:
# Create the Bitcoin sentiment scores DataFrame
btc_sentiment = get_sent_df(btc_news)
btc_sentiment.head()

,date,text,compound,positive,negative,neutral
0,2021-06-05,"Hello friends, and welcome back to Week in Rev...",0.7500,0.154,0.000,0.846
1,2021-06-08,"By Mary-Ann RussonBusiness reporter, BBC News\...",0.3400,0.070,0.000,0.930
2,2021-06-03,"Cryptocurrencies such as bitcoin are a ""farce""...",0.0000,0.000,0.000,1.000
3,2021-06-05,"In a video broadcast to Bitcoin 2021, a multid...",0.0000,0.000,0.000,1.000
4,2021-06-03,This story originally appeared on DueTheft and...,0.4588,0.160,0.113,0.728


In [8]:
# Create the Ethereum sentiment scores DataFrame
eth_sentiment = get_sent_df(eth_news)

In [9]:
# Describe the Bitcoin Sentiment
btc_sentiment.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.240275,0.082950,0.027700,0.889350
std,0.336668,0.093365,0.040175,0.115746
min,-0.292400,0.000000,0.000000,0.582000
25%,0.000000,0.000000,0.000000,0.828250
50%,0.077150,0.072500,0.000000,0.908000
75%,0.497200,0.155500,0.065250,1.000000
max,0.921700,0.351000,0.113000,1.000000


In [10]:
# Describe the Ethereum Sentiment
eth_sentiment.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.00000,20.000000
mean,0.198135,0.067750,0.01800,0.914250
std,0.341426,0.076853,0.04171,0.095173
min,-0.458800,0.000000,0.00000,0.655000
25%,0.000000,0.000000,0.00000,0.849000
50%,0.064000,0.054000,0.00000,0.928500
75%,0.477750,0.126000,0.00000,1.000000
max,0.750000,0.246000,0.15100,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: btc had a higher mean positive score.

Q: Which coin had the highest compound score?

A: btc had a higher compound score.

Q. Which coin had the highest positive score?

A: btc had a higher positive score.

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [11]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [12]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()
# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [14]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Create a tokenized list of the words
    words = word_tokenize(text)
   
    # Convert the words to lowercase
    
    words = list(filter(lambda word: word.lower(), words))
   # lc_words = [word.lower() for word in words if word.lower() not in sw]
    
    # Remove the punctuation from text
    words = list(filter(lambda p: p not in punctuation, words))
    
    # Remove the stop words
    words = list(filter(lambda word: word not in sw, words))
    
    # Lemmatize words into root words
    tokens = list(filter(lambda word: lemmatizer.lemmatize(word), words))
    #tokens = [lemmatizer.lemmatize(word) for word in words]
    return tokens

In [15]:
# Create a new tokens column for Bitcoin
btc_sentiment["tokens"] = btc_sentiment["text"].apply(tokenizer)
btc_sentiment.head()

,date,text,compound,positive,negative,neutral,tokens
0,2021-06-05,"Hello friends, and welcome back to Week in Rev...",0.7500,0.154,0.000,0.846,"[Hello, friends, welcome, back, Week, Review, ..."
1,2021-06-08,"By Mary-Ann RussonBusiness reporter, BBC News\...",0.3400,0.070,0.000,0.930,"[By, Mary-Ann, RussonBusiness, reporter, BBC, ..."
2,2021-06-03,"Cryptocurrencies such as bitcoin are a ""farce""...",0.0000,0.000,0.000,1.000,"[Cryptocurrencies, bitcoin, ``, farce, '', sym..."
3,2021-06-05,"In a video broadcast to Bitcoin 2021, a multid...",0.0000,0.000,0.000,1.000,"[In, video, broadcast, Bitcoin, 2021, multiday..."
4,2021-06-03,This story originally appeared on DueTheft and...,0.4588,0.160,0.113,0.728,"[This, story, originally, appeared, DueTheft, ..."


In [16]:
# Create a new tokens column for Ethereum
eth_sentiment["tokens"] = eth_sentiment["text"].apply(tokenizer)
eth_sentiment.head()

,date,text,compound,positive,negative,neutral,tokens
0,2021-06-03,This new mining feature is called 'Norton Cryp...,0.0000,0.000,0.000,1.000,"[This, new, mining, feature, called, 'Norton, ..."
1,2021-06-04,CryptoPunks represent the only historically re...,-0.4588,0.067,0.151,0.782,"[CryptoPunks, represent, historically, relevan..."
2,2021-06-01,"Yao Qian, the former head of China's digital y...",0.3612,0.070,0.000,0.930,"[Yao, Qian, former, head, China, 's, digital, ..."
3,2021-06-01,A consumer uses a digital yuan red envelope in...,0.0000,0.000,0.000,1.000,"[A, consumer, uses, digital, yuan, red, envelo..."
4,2021-06-03,One of the biggest challenges for developers b...,0.4588,0.118,0.000,0.882,"[One, biggest, challenges, developers, buildin..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [17]:
from collections import Counter
from nltk import ngrams

In [29]:
def gen_bigrams(tokens): 
    bigrams = ngrams(tokens, n=2)
    return list(bigrams)

In [45]:
# Generate the Bitcoin N-grams where N=2
btc_tokens = btc_sentiment["tokens"].copy()
btc_bigrams = btc_tokens.apply(gen_bigrams)

In [51]:
zip(btc_bigrams)
# map(list,zip(btc_tokens))
btc_bigrams

0     [(Hello, friends), (friends, welcome), (welcom...
1     [(By, Mary-Ann), (Mary-Ann, RussonBusiness), (...
2     [(Cryptocurrencies, bitcoin), (bitcoin, ``), (...
3     [(In, video), (video, broadcast), (broadcast, ...
4     [(This, story), (story, originally), (original...
5     [(The, cryptocurrency), (cryptocurrency, rolle...
6     [(Federal, agents), (agents, tracked), (tracke...
7     [(By, Reuters), (Reuters, Staff), (Staff, MEXI...
8     [(Representations, Bitcoin), (Bitcoin, cryptoc...
9     [(Bitcoin, NurPhoto/Getty), (NurPhoto/Getty, I...
10    [(Bitcoin, slipped), (slipped, 3), (3, Friday)...
11    [(Small, toy), (toy, figurines), (figurines, s...
12    [(Blockchain, technology), (technology, compan...
13    [(Bitcoin, mining), (mining, uses), (uses, hug...
14    [(A, representation), (representation, virtual...
15    [(This, article), (article, translated), (tran...
16    [(A, representation), (representation, virtual...
17    [(The, crypto), (crypto, world), (world, s

In [44]:
# btc_all = np.concatenate(btc_bigrams.copy())
# btc_all

In [66]:
# Generate the Ethereum N-grams where N=2
eth_bigrams = eth_sentiment["tokens"].apply(gen_bigrams)
eth_bigrams

0     [(This, new), (new, mining), (mining, feature)...
1     [(CryptoPunks, represent), (represent, histori...
2     [(Yao, Qian), (Qian, former), (former, head), ...
3     [(A, consumer), (consumer, uses), (uses, digit...
4     [(One, biggest), (biggest, challenges), (chall...
5     [(By, Reuters), (Reuters, Staff), (Staff, FILE...
6     [(Representations, cryptocurrencies), (cryptoc...
7     [(After, almost), (almost, 8), (8, years), (ye...
8     [(This, week), (week, SpaceX), (SpaceX, launch...
9     [(While, almost), (almost, certainly), (certai...
10    [(Los, NFT), (NFT, plantean), (plantean, una),...
11    [(The, cryptocurrency), (cryptocurrency, rolle...
12    [(Ethereum, hot), (hot, commodity), (commodity...
13    [(Representations, cryptocurrencies), (cryptoc...
14    [(A, representation), (representation, virtual...
15    [(Standard, Chartered), (Chartered, hopping), ...
16    [(El, negocio), (negocio, de), (de, las), (las...
17    [(Hello, friends), (friends, welcome), (we

In [67]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [75]:
# Use token_count to get the top 10 words for Bitcoin
btc_10 = btc_sentiment["tokens"].apply(token_count)
btc_10 = pd.DataFrame(list(btc_10.items()),columns=["word", "count"])
btc_10

,word,count
0,0,"[(week, 2), (I, 2), (Hello, 1), (friends, 1), ..."
1,1,"[(Cryptocurrencies, 1), (bitcoin, 1), (``, 1),..."
2,2,"[(This, 1), (story, 1), (originally, 1), (appe..."
3,3,"[(The, 1), (cryptocurrency, 1), (rollercoaster..."
4,4,"[(Bitcoin, 1), (slipped, 1), (3, 1), (Friday, ..."
5,5,"[(Small, 1), (toy, 1), (figurines, 1), (seen, ..."
6,6,"[(Blockstream, 2), (Blockchain, 1), (technolog..."
7,7,"[(Bitcoin, 1), (mining, 1), (uses, 1), (huge, ..."
8,8,"[(The, 2), (Fine, 1), (Print, 1), (following, ..."
9,9,"[(A, 1), (representation, 1), (virtual, 1), (c..."


In [72]:
# Use token_count to get the top 10 words for Ethereum
eth_10 = eth_sentiment["tokens"].apply(token_count)
eth_10

0     [(Norton, 3), (Crypto, 2), (This, 1), (new, 1)...
1     [(CryptoPunks, 1), (represent, 1), (historical...
2     [(digital, 2), (Yao, 1), (Qian, 1), (former, 1...
3     [(digital, 3), (yuan, 2), ('s, 2), (A, 1), (co...
4     [(One, 1), (biggest, 1), (challenges, 1), (dev...
5     [(By, 1), (Reuters, 1), (Staff, 1), (FILE, 1),...
6     [(Representations, 1), (cryptocurrencies, 1), ...
7     [(After, 1), (almost, 1), (8, 1), (years, 1), ...
8     [(SpaceX, 2), (ETH, 2), (node, 2), (This, 1), ...
9     [(new, 2), (While, 1), (almost, 1), (certainly...
10    [(una, 2), (del, 2), (Los, 1), (NFT, 1), (plan...
11    [(The, 1), (cryptocurrency, 1), (rollercoaster...
12    [(Ethereum, 1), (hot, 1), (commodity, 1), (cry...
13    [(Representations, 1), (cryptocurrencies, 1), ...
14    [(A, 1), (representation, 1), (virtual, 1), (c...
15    [(Standard, 1), (Chartered, 1), (hopping, 1), ...
16    [(de, 3), (el, 2), (El, 1), (negocio, 1), (las...
17    [(week, 2), (I, 2), (Hello, 1), (friends, 

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---